
# Data

Buscamos primero la data que dejamos por github, hay banda asi que eso nos va a llevar unas lineas.
Todo disponible en: https://github.com/agustring/LSA_classifier

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf

l = '001'
u = '004'
s = '004'
url = 'https://raw.githubusercontent.com/agustring/LSA_classifier/main/preproc%20data/{}_{}_{}_pre.json?token=ARECJQ4LDX673DAOGWXLV5TBSKZ5W'.format(l, u, s)

df = pd.read_json(url)
df.shape #Forma de un video (None,43,3)

(53, 43)

In [11]:
import time
import urllib.request, json 

t1 = time.time()
x=0

dataset = []
data_for_visual = np.zeros(750,dtype=object)

def help_the_norman(df):
  a = df
  min_i = 0
  min_j = 0
  for i in range(len(df)):
    j = 10
    if df[i][j][0] < 1e-6:
      min_i = i
    if df[i][j+20][0] < 1e-6:
      min_j = i
    if min_j+1 > len(df[i][j]):
      min_j = 0
    if min_i+1 > len(df[i][j]):
      min_i = 0
  #print(min_i)

  for i in range(int(min_i)):
    for j in range(22):
      a[i][j][0] = df[min_i+1][j][0]
      a[i][j][1] = df[min_i+1][j][1]

  for i in range(int(min_j)):
    for j in range(22,42):
      a[i][j][0] = df[min_j+1][j][0]
      a[i][j][1] = df[min_j+1][j][1]

  return a

def norman(df):
  a = df
  min_x = df[0][0][0]
  max_x = df[0][0][0]
  min_y = df[0][0][1]
  max_y = df[0][0][1]
  for i in range(len(df)):
    for j in range(len(df[i])):
      if df[i][j][0] < min_x:
        min_x = df[i][j][0]
      if df[i][j][0] > max_x:
        max_x = df[i][j][0]
      if df[i][j][1] < min_y:
        min_y = df[i][j][1]
      if df[i][j][1] > max_y:
        max_y = df[i][j][1]

  for i in range(len(df)):
    for j in range(len(df[i])):
      b = (max_x-min_x)
      c = (max_y-min_y)
      if b > 1e-6:
        a[i][j][0] = (df[i][j][0]-min_x)/b
        a[i][j][1] = (df[i][j][1]-min_y)/c
      else:
        a[i][j][0] = 0
        a[i][j][1] = 0
  return a

for user in range(1,11): #1,11
  u = '00'+str(user)
      
  if user>9:
      u = '0'+str(user)

  for label in range(1,16): #1,16
      
      l = '00'+str(label)
      
      if label>9:
          l = '0'+str(label)
          
      for sample in range(1,6): #1,6
          t2 = time.time()
          s = '00'+str(sample)
          
          url = 'https://raw.githubusercontent.com/agustring/LSA_classifier/main/preproc%20data2/{}_{}_{}_pre.json?token=ARECJQ4LDX673DAOGWXLV5TBSKZ5W'.format(u, l, s)
          
          with urllib.request.urlopen(url) as url2:
              df = json.loads(url2.read().decode())
          df = norman(df)
          df = help_the_norman(df)
          data_struct = np.array([[0 for i in range(2*42)] for j in range(150)], dtype=float)
          for i in range(len(df)):
              p=0
              for j in range(0,42,2):
                  data_struct[i+(150-len(df)),j] = df[i][p][0]
                  data_struct[i+(150-len(df)),j+42] = df[i][p][1]
                  #data_struct[i,j+2] = df[i][p][2]
                  p+=1
          dataset.append(data_struct)
          data_for_visual[x] = np.array(df)

print('Todo en minutos: ',(time.time()-t1)/60)
            
dataset = np.array(dataset)

Todo en minutos:  6.535082240899404


In [12]:
print(dataset[3])
print(dataset[3][1])
print(dataset[3][1][1])

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.5108453 0.        0.5006721 ... 0.        0.6351324 0.       ]
 [0.5108453 0.        0.5006721 ... 0.        0.6351324 0.       ]
 [0.5108453 0.        0.5006721 ... 0.        0.6351324 0.       ]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


In [13]:
labels = np.zeros(len(dataset),dtype=int)
x=-1
for k in range(0,len(dataset),5):
    x+=1
    if x==15:
        x=0
    labels[k:k+5] = int(x)
    
labels = np.array(labels)

In [14]:
import random

tst = .5
index = []
index_lbl = []

for i in range(0,int(len(dataset)),5):
  index_lbl.append(i+2)
  index.append(i)
  index.append(i+1)
  index.append(i+3)
  index.append(i+4)

random.shuffle(index)
random.shuffle(index_lbl)

X_train = dataset[index]
y_train = labels[index]
x_test = dataset[index_lbl]
y_test = labels[index_lbl]


#from sklearn.model_selection import train_test_split

#X_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

# Ahora si 😈

Esto es viejo, ahora es `input_shape=(750, 150, 43* 2)` 

`batch_input_shape=(750, None, 43, 3)` 

Bueno al final no me dejo poner 750

Los 750 son los videos.

El None la longitud de cada uno (indeterminada).

El 43 los atributos.

El ultimo 3 las dimensiones de la mano.


In [15]:
print(X_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(600, 150, 84)
(600,)
(150, 150, 84)
(150,)


In [ ]:
os.listdir(checkpoint_dir)

# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

In [ ]:
import numpy as np
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM, Flatten, Embedding, Input, Conv1D,MaxPooling1D,Bidirectional
from tensorflow.keras.optimizers import Adam, Adamax, SGD, RMSprop
from keras import metrics
import os

checkpoint_path = "training_1/lsa_classifier.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=False)

history = []
with tpu_strategy.scope():
  model = Sequential()
model.add(Input((150,84)))
model.add(Bidirectional(LSTM(84*7, activation="tanh",recurrent_activation="sigmoid", input_shape=(150,420),return_sequences=False)))
model.add(Dense(15, activation='softmax'))
opt = Adamax(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax")
#opt = RMSprop(learning_rate=0.001,rho=0.9,momentum=0.0,epsilon=1e-07,centered=False, name="RMSprop")
#opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

#with tf.device('/device:GPU:0'):

history.append(model.fit(X_train, y_train, 
                          epochs=1500, batch_size=10,
                          validation_data=(x_test, y_test)))
                          #callbacks=[cp_callback]))
  
y_pred = model.predict(x_test)
#with tf.device('/device:GPU:0'):
#model.add(Conv1D(filters=84, kernel_size=3, activation='relu', input_shape=(150,84)))
#model.add(Conv1D(filters=84*10, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=84*50, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=84*150, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
#model.add(MaxPooling1D(pool_size=3))
#model.add(Bidirectional(layers.LSTM(84*5,  activation="tanh",recurrent_activation="sigmoid", input_shape=(150,84),return_sequences=True)))
#model.add(LSTM(84*5,  activation="tanh",recurrent_activation="sigmoid", input_shape=(150,84),return_sequences=True))

#model.add(Flatten())
#model.add(Dense(500, activation='relu'))
#opt = RMSprop(learning_rate=0.001,rho=0.9,momentum=0.0,epsilon=1e-07,centered=False, name="RMSprop")
#opt = Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")\
#opt = SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")


In [190]:
i=33
print(y_pred[i])
print(y_pred[i].max() == (y_pred[i][y_test[i]]))
print(y_test[i])

[1.2507070e-03 8.7476394e-05 4.6626028e-06 7.8377007e-03 9.1016853e-05
 4.2175705e-04 1.7380656e-03 5.6926045e-03 1.7873926e-02 9.4471496e-01
 8.7784556e-06 2.5980317e-03 1.6878217e-02 3.4460006e-04 4.5748890e-04]
True
9


In [ ]:
pred = []
for i in range(len(y_pred)):
  pred.append(y_pred[i].max() == (y_pred[i][y_test[i]]))
print(np.count_nonzero(pred)/len(y_pred))

In [27]:
history[0]

In [ ]:
fig, axs = plt.subplots(len(history)+1)
fig.set_size_inches(10,8*len(history)+1)
fig.suptitle('Varias muestras')

for video_num in range(len(history)):
  axs[video_num].set_title("Epochs=40; LSTM nodes: "+str((video_num+7)*84))
  axs[video_num].set_xlabel('x')
  axs[video_num].set_ylabel('y')
  #axs[video_num].axis(xmin=0,xmax=1,ymin=0,ymax=1)
  axs[video_num].plot(history[video_num].history['accuracy'])
  axs[video_num].plot(history[video_num].history['val_accuracy'])

  fig.show()

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import keras
from matplotlib import pyplot as plt
history = model1.fit(train_x, train_y,validation_split = 0.1, epochs=50, batch_size=4)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()